# UFO Question 

Our data science team has predicted that the Earth is going to be invaded by an alien force in the
next years. Our only hope is to replicate a device that can block all alien technology in a radius of
~300km. Sadly, the device was sold in 2004 to an anonymous buyer to protect her hometown and
we don't know how contact her again. We know that the device has been active since 2004 in one
city in the USA, and we want to know where to start our search.
We've included a dataset called ​ufo.csv​. This dataset contains over 80,000 reports of UFO sightings
over the last century (all of them verified by the ESA). Using this dataset, try to guess the city in
which the device has been hidden.


In [1]:
%pip install pymongo

from pymongo import MongoClient
import pandas as pd
import re 

from datetime import datetime


Note: you may need to restart the kernel to use updated packages.


In [2]:

from datetime import datetime

In [3]:
cliente=MongoClient()
db=cliente.ufo

In [4]:
cliente.list_database_names() 

['admin', 'companies', 'config', 'local', 'ufo']

In [5]:
!pip install folium
import folium

In [6]:
cliente=MongoClient()

db=cliente.ufo
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ufo')

In [7]:
data=pd.read_csv(r'ufo.csv')
data.head()

,Unnamed: 0,datetime,city,state,country,shape,duration,total_time,comments,date_posted,latitude,longitude,year,distance
0,0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,2004,1242.667772
1,1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,2005,1325.486319
2,2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,2008,6515.416577
3,3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,2004,1211.971352
4,4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,2004,6960.923396


In [26]:
ufo=data.copy()

ufo=ufo[(ufo.country=="us")]


ufo


ValueError: cannot insert geo_loc, already exists

In [9]:

#ufo.datetime=pd.to_datetime(ufo1.datetime,format='%Y-%m-%d %H:%M')

In [10]:

ufo=ufo.drop(['shape','state','duration','total_time','Unnamed: 0','date_posted','comments','year'], axis=1)
ufo

,datetime,city,country,latitude,longitude,distance
0,10/10/1949 20:30,san marcos,us,29.883056,-97.941111,1242.667772
3,10/10/1956 21:00,edna,us,28.978333,-96.645833,1211.971352
4,10/10/1960 20:00,kaneohe,us,21.418056,-157.803611,6960.923396
5,10/10/1961 19:00,bristol,us,36.595000,-82.188889,427.334113
7,10/10/1965 23:45,norwalk,us,41.117500,-73.408333,1295.827792
...,...,...,...,...,...,...
78504,9/9/2013 21:00,woodstock,us,34.101389,-84.519444,318.719195
78505,9/9/2013 22:00,boise,us,43.613611,-116.202500,2620.021539
78506,9/9/2013 22:00,napa,us,38.297222,-122.284444,3126.991417
78507,9/9/2013 22:20,vienna,us,38.901111,-77.265556,904.740389


In [11]:
ufo.reset_index(inplace=True)

In [12]:

#ufo_index=ufo_index.drop(['level_0','index'], axis=1)
#ufo['datetime'] = pd.DatetimeIndex(ufo['datetime']).year

ufo.head()

,index,datetime,city,country,latitude,longitude,distance
0,0,10/10/1949 20:30,san marcos,us,29.883056,-97.941111,1242.667772
1,3,10/10/1956 21:00,edna,us,28.978333,-96.645833,1211.971352
2,4,10/10/1960 20:00,kaneohe,us,21.418056,-157.803611,6960.923396
3,5,10/10/1961 19:00,bristol,us,36.595000,-82.188889,427.334113
4,7,10/10/1965 23:45,norwalk,us,41.117500,-73.408333,1295.827792


In [13]:
def data():
    ufo.insert(2, 'geo_loc', pd.Series(list(range(len(ufo)))))
    ufo['geo_loc']=ufo.apply(lambda x: [x['longitude'], x['latitude']], axis=1)
    
    return ufo

In [14]:
ufo=data()
print('numero total de compañias consideras:', len(ufo))

numero total de compañias consideras: 63399


In [15]:
db.geoloc.insert_many(ufo.to_dict('records'))

In [16]:
db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [17]:
#ufo.datetime=ufo.datetime.str.extract(r'(\\d{4})')
ufo.head()

,index,datetime,geo_loc,city,country,latitude,longitude,distance
0,0,10/10/1949 20:30,"[-97.9411111, 29.8830556]",san marcos,us,29.883056,-97.941111,1242.667772
1,3,10/10/1956 21:00,"[-96.6458333, 28.9783333]",edna,us,28.978333,-96.645833,1211.971352
2,4,10/10/1960 20:00,"[-157.8036111, 21.4180556]",kaneohe,us,21.418056,-157.803611,6960.923396
3,5,10/10/1961 19:00,"[-82.1888889, 36.595]",bristol,us,36.595000,-82.188889,427.334113
4,7,10/10/1965 23:45,"[-73.4083333, 41.1175]",norwalk,us,41.117500,-73.408333,1295.827792


In [27]:
#ufo.datetime=ufo.datetime.astype(int)
ufo = ufo[(ufo.distance > 290) & (ufo.distance < 310)]
ufo.reset_index(drop = True, inplace = True)

ufo

,index,datetime,geo_loc,city,country,latitude,longitude,distance
0,12,10/10/1970 19:00,"[-83.7619444, 37.1536111]",manchester,us,37.153611,-83.761944,302.476051
1,151,10/10/2006 12:37,"[-83.9583333, 34.8761111]",blairsville,us,34.876111,-83.958333,305.862395
2,877,10/1/2009 18:00,"[-90.0488889, 35.1494444]",memphis,us,35.149444,-90.048889,305.272767
3,964,10/12/1973 19:00,"[-86.8025, 33.5205556]",birmingham (just south of),us,33.520556,-86.802500,295.975266
4,1192,10/13/1969 22:00,"[-90.0488889, 35.1494444]",memphis,us,35.149444,-90.048889,305.272767
...,...,...,...,...,...,...,...,...
223,76230,9/27/2002 19:51,"[-90.0488889, 35.1494444]",memphis,us,35.149444,-90.048889,305.272767
224,76354,9/27/2013 19:35,"[-86.8113889, 33.4052778]",hoover,us,33.405278,-86.811389,308.723830
225,76733,9/29/2011 21:00,"[-90.0488889, 35.1494444]",memphis,us,35.149444,-90.048889,305.272767
226,76847,9/30/2000 20:00,"[-90.0488889, 35.1494444]",memphis,us,35.149444,-90.048889,305.272767


In [19]:
def geoquery(ufo):
    

    
    for i in range(len(ufo)):
        
        q=list(
            ufo.geoloc.find(
                {'geo_loc': {'$near':{'$geometry':{
                    'type':'Point',
                    'coordinates':[ufo.longitude[i], ufo.latitude[i]]
                },
                                     '$maxDistance':310}}}
            )
        )
        
    return ufo

In [20]:
def geomap(ufo, zoom_start=1.5):

    ufo.index=range(len(ufo)) 

    mapa=folium.Map(location=[0, 0],
                tiles='openstreetmap', zoom_start=zoom_start)

    for i in range(1, len(ufo)):

        folium.CircleMarker([float(ufo.latitude[i]), float(ufo.longitude[i])], popup=str(ufo.city[i]), 
                             radius=1, icon=folium.Icon()).add_to(mapa)

    mapa.save('images/geomap.html')
    
    
    return mapa
            

In [25]:
ufo=geoquery(ufo)
ufo.head()

AttributeError: 'DataFrame' object has no attribute 'geoloc'

In [28]:
mapa=geomap(ufo.loc[:228])
mapa